In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from nltk import word_tokenize, ngrams 
from sklearn import ensemble
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
import xgboost as xgb
eng_stopwords = set(stopwords.words('english'))
from pandas import DataFrame

d:\python35\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# 导入特征文件

In [2]:
train1 = pd.read_csv("../train/train1.csv")
test1 = pd.read_csv("../test/test1.csv")
print("train1:",train1.shape)
print("test1:",test1.shape)

train2 = pd.read_csv("../train/train2.csv")
test2 = pd.read_csv("../test/test2.csv")
print("train2:",train2.shape)
print("test2:",test2.shape)

train3 = pd.read_csv("../train/train3.csv")
test3 = pd.read_csv("../test/test3.csv")
print("train3:",train3.shape)
print("test3:",test3.shape)

train4 = pd.read_csv("../train/train4.csv")
test4 = pd.read_csv("../test/test4.csv")
print("train4:",train4.shape)
print("test4:",test4.shape)

train1: (404290, 8)
test1: (2345796, 5)
train2: (404290, 7)
test2: (2345796, 7)
train3: (404290, 6)
test3: (2345796, 5)
train4: (404290, 2)
test4: (2345796, 2)


In [3]:
train3=train3[['id','q1_freq','q2_freq']]

In [4]:
train_df=pd.merge(train1,train2,how='inner', on='id') 
train_df=pd.merge(train_df,train3,how='inner', on='id') 
train_df=pd.merge(train_df,train4,how='inner', on='id') 

In [5]:
#train3=train3[['id','q1_freq','q2_freq']]
test3.rename(columns={'id':'test_id'},inplace=True)
test3=test3[['test_id','q1_freq','q2_freq']]

In [6]:
test_df=pd.merge(test1,test2,how='inner', on='test_id') 
test_df=pd.merge(test_df,test3,how='inner', on='test_id') 
test_df=pd.merge(test_df,test4,how='inner', on='test_id') 

In [7]:
train_df.drop(['qid1','qid2','question1','question2'],axis=1, inplace=True)
test_df.drop(['question1','question2'],axis=1, inplace=True)

In [8]:
print("train_df:",train_df.shape)
print("test_df",test_df.shape)

train_df: (404290, 13)
test_df (2345796, 12)


In [10]:
label=train_df[['id','is_duplicate']]
train_df=train_df.drop(['id','is_duplicate'],axis=1)

In [11]:
train_df.fillna(0,inplace=True)#404290 rows × 11 columns
train_df.head()

,word_match,tfidf_word_match,common_unigrams_len,common_unigrams_ratio,common_bigrams_len,common_bigrams_ratio,common_trigrams_len,common_trigrams_ratio,q1_freq,q2_freq,q1_q2_intersect
0,0.909091,0.972370,5.0,0.833333,5.0,0.833333,4.0,0.800000,1,2,0
1,0.588235,0.641837,5.0,0.416667,5.0,0.416667,4.0,0.363636,8,3,0
2,0.363636,0.344503,2.0,0.222222,0.0,0.000000,0.0,0.000000,2,1,0
3,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,1,1,0
4,0.266667,0.262659,2.0,0.153846,0.0,0.000000,0.0,0.000000,3,1,0


In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(interaction_only=True)
train_df=poly.fit_transform(train_df)
train_df = pd.DataFrame(train_df.reshape(404290,67))
train_df=pd.concat([label,train_df],axis=1)

In [18]:
label=test_df[['test_id']]
test_df=test_df.drop(['test_id'],axis=1)
test_df.fillna(0,inplace=True)#2345796 rows × 11 columns

d:\python35\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
d:\python35\lib\site-packages\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
d:\python35\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains

In [ ]:
test_df = pd.DataFrame(test_df.reshape(2345796,67))
test_df=pd.concat([label,test_df],axis=1)
test_df.drop([0],axis=1,inplace=True)
train_df.drop([0],axis=1,inplace=True)

In [23]:
x_train=train_df
y_train = train_df['is_duplicate'].values
print("train_df:",train_df.shape)
print("test_df",test_df.shape)
print("x_train:",x_train.shape)
print("y_train",len(y_train))

train_df: (404290, 68)
test_df (2345796, 67)
x_train: (404290, 68)
y_train 404290


In [25]:
x_train.drop(['id'],axis=1, inplace=True)
sub = pd.DataFrame()
sub['test_id'] = test_df['test_id']
x_test=test_df.drop(['test_id'],axis=1)
print("x_train:",x_train.shape)
print("y_train",len(y_train))

# XGBoost

In [27]:
from sklearn.cross_validation import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=4242)
print("x_train:",x_train.shape)
print("y_train",len(y_train))
print("x_valid:",x_valid.shape)
print("y_valid",len(y_valid))
print("x_test:",x_test.shape)

x_train: (323432, 67)
y_train 323432
x_valid: (80858, 67)
y_valid 80858
x_test: (2345796, 66)


In [28]:
x_train.drop(['is_duplicate'],axis=1, inplace=True)
x_valid.drop(['is_duplicate'],axis=1, inplace=True)

d:\python35\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
d:\python35\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [ ]:
import xgboost as xgb
#XGBoost自带接口
params={
    'eta': 0.1,
    'max_depth':6,   
    'min_child_weight':1,
    'gamma':0.3, 
    'subsample':0.8,
    'colsample_bytree':0.8,
    'booster':'gbtree',
    'objective': 'binary:logistic',
    'nthread':12,
    'scale_pos_weight': 1,
    'lambda':1,  
    'seed':27,
    'silent':0 ,
    'eval_metric': 'logloss'
}

d_train = xgb.DMatrix(x_train, label=y_train)
d_valid = xgb.DMatrix(x_valid, label=y_valid)
d_test = xgb.DMatrix(x_test)

watchlist = [(d_train, 'train'), (d_valid, 'valid')]

bst = xgb.train(params, d_train, 3000, watchlist, early_stopping_rounds=500, verbose_eval=10)

In [ ]:
bst.best_iteration

In [ ]:
y_bst= bst.predict(d_test)
sub['is_duplicate'] = y_bst
sub

In [ ]:
sub.to_csv('../output/xgb_20170521_B.csv', index=False)

# 模型

In [ ]:
output1=pd.read_csv('../output/0.2613_lstm_215_108_0.19_0.20.csv')
output2=pd.read_csv('../output/0.2662_lstm_232_118_0.39_0.15.csv')
output3=pd.read_csv('../output/0.2713_lstm_208_112_0.39_0.27.csv')
output4=pd.read_csv('../output/0.2723_lstm_190_136_0.29_0.27.csv')

output=output1[['test_id']]

x1=output1.is_duplicate
x2=output2.is_duplicate
x3=output3.is_duplicate
x4=output4.is_duplicate

output['is_duplicate']=0.25*x1+0.25*x2+0.25*x3+0.25*x4
output.to_csv('../output/Avg_20170521_A.csv', index=False)#0.28687

# 加入其它特征